## MOD - Negative Feedback - Mass Action

<img src="../illustrations/Motif__Negative_Feedback.png" alt="Negative Feedback Motif" width="300" align="left" margin="2000" style="padding-right:50px"/>


#### Equations

$\frac{dA}{dt} = k_{IA} \cdot I \cdot (1 - A) - k_{BA} \cdot B \cdot A - k_{Ad} \cdot A$

$\frac{dB}{dt} = k_{AB} \cdot A \cdot (1 -B) - k_{Bd} \cdot B$

### Prep

In [ ]:
# Imports

import os

import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import odeint
from scipy.optimize import minimize
from scipy.optimize import basinhopping, brute

from ipywidgets import interact

import sys; sys.path.insert(0, '..')
import optonotch.utilities as utils
import optonotch.modeling as md

In [ ]:
# Load and prep the data

dpath = '../Data/Measurements/Sim/compiled_count_data.csv'

# Load data from compiled data file, which was
# generated in `ANA - sim spot detection.ipynb`
data = md.data_loading.load_data(dpath)

# Uniformly truncate the end to simplify things
data = md.data_loading.truncate_exp(data)

# Rescale intensity to avoid dealing with large
# numbers during MSE minimization;
# doesn't change anything as the raw scale is
# arbitrary to begin with
data = md.data_loading.rescale_intensity(data)

### Model Definition

In [ ]:
### Define the model

def ode_model(X, t, params, input_func):
    """Negative Feedback Adaptation (Mass Action)"""
    
    A, B = X
    k_IA, k_BA, k_Ad, k_AB, k_Bd = params
    I = input_func(t)
    
    dA = k_IA * I * (1-A)  -  k_BA * B * A  - k_Ad * A
    dB = k_AB * A * (1-B)  -  k_Bd * B

    return (dA, dB)

In [ ]:
### Show model with initial guess

# Parameters (initial guess)
k_IA = 1
k_BA = 1
k_Ad = 0.1
k_AB = 0.1
k_Bd = 0.1
params = (k_IA, k_BA, k_Ad, k_AB, k_Bd)

# Initial conditions
A = 0.0
B = 0.0
ini = (A, B)

# Time
dt = 0.1
t = np.arange(0, 40, dt)

# Solve
solution = odeint(ode_model, ini, t, 
                  args=(params, md.input_functions.input_continuous_model))
A  = solution[:,0]
B  = solution[:,1]

# Plot result
fig, ax = plt.subplots(3, 1, figsize=(10, 6), sharex=True)

ax[0].plot(t, [md.input_functions.input_continuous_model(t_) for t_ in t], '-', color='b', lw=1)
ax[0].set_ylabel('Input', fontsize=14)

ax[1].plot(t, B, '-', color='g', lw=1)
ax[1].set_ylabel('B', fontsize=14)

ax[2].plot(t, A, '-', color='r', lw=1)
ax[2].set_ylabel('Output (A)', fontsize=14)

plt.xlabel('time [a.u.]', fontsize=14)

plt.tight_layout()
plt.show()

### Model Fitting

In [ ]:
### Run basinhopping optimization

# Bounds on parameters
k_IA_bounds = (1e-4, 100)
k_BA_bounds = (1e-4, 100)
k_Ad_bounds = (1e-4, 100)
k_AB_bounds = (1e-4, 100)
k_Bd_bounds = (1e-4, 100)
param_bounds = [k_IA_bounds, k_BA_bounds, k_Ad_bounds, 
                k_AB_bounds, k_Bd_bounds]

# Select & flatten the data
data_times  = np.concatenate(data['continuous']['times'])
#data_values = np.concatenate(data['continuous']['counts'])
data_values = np.concatenate(data['continuous']['ints'])
#scale_factor = 250
scale_factor = 100

# Optimization
res = basinhopping(md.loss_functions.mse_loss, params, niter=100, stepsize=0.5, seed=42, 
                   minimizer_kwargs={'args'   : (ode_model, ini, t, 
                                                 md.input_functions.input_continuous_model, 
                                                 data_times, data_values, scale_factor),
                                     'bounds' : param_bounds}
                   )
optimal_params = res.x

# Report
print('Message:', res.message)
print(f'Final params:\n  k_IA={res.x[0]:.4f}\n  k_BA={res.x[1]:.4f}\n  k_Ad={res.x[2]:.4f}'+
      f'\n  k_AB={res.x[3]:.4f}\n  k_Bd={res.x[4]:.4f}')
print('Final loss:', res.fun)

In [ ]:
### Plot the results with both continuous and pulsatile input

### Prep

# Recreate simulations to visualize
solution_early = odeint(ode_model, ini, t, 
                        args=(optimal_params, md.input_functions.input_continuous_model))
solution_early *= scale_factor
solution_pulse = odeint(ode_model, ini, t, 
                        args=(optimal_params, md.input_functions.input_pulsatile_model))
solution_pulse *= scale_factor

# Initialize figure
fig,ax = plt.subplots(2, 1, figsize=(7, 6), sharex=True)
ax[0].set_title('Negative feedback', fontsize=15, fontname='Arial')


### Continuous

# Input
ax[0].plot(t, np.array([md.input_functions.input_continuous_model(tx) for tx in t])*scale_factor, 
           '-.', color='#0F76BB', label='input', zorder=1, lw=1.5, alpha=1.0)

# Data
first = True
for plot_x, plot_y in zip(data['continuous']['times'], data['continuous']['ints']):
    ax[0].plot(plot_x, plot_y, 
               c='teal', alpha=0.7,
               label='data' if first else '__no_label__', zorder=2)
    first=False

# Model
ax[0].plot(t, solution_early[:,0], '-', color='k', lw=2.0, label='model', zorder=3)

# Cosmetics
ax[0].set_ylabel('$sim$ expression', fontsize=14, fontname='Arial', labelpad=0)
ax[0].tick_params(axis='both', which='major', labelsize=14)
[tick.set_fontname('Arial') for tick in ax[0].get_xticklabels()]
[tick.set_fontname('Arial') for tick in ax[0].get_yticklabels()]

handles, labels = ax[0].get_legend_handles_labels()
order = [0,1,2]
ax[0].legend([handles[idx] for idx in order[:len(handles)]], 
             [labels[idx] for idx in order[:len(labels)]],
             scatterpoints=3, loc=(0.8,0.40), fontsize=12,
             prop={'family':'Arial', 'size':12})


### Pulsatile

# Prep
t_end_p = 32.0
t_p = t[t<=t_end_p]

# Input
ax[1].plot(t_p, np.array([md.input_functions.input_pulsatile_model(tx) for tx in t_p])*scale_factor, 
           '-.', color='#0F76BB', label='input', zorder=1, lw=1.5, alpha=0.7)

# Data
first = True
for plot_x, plot_y in zip(data['pulsatile']['times'], data['pulsatile']['ints']):
    ax[1].plot(plot_x[:10], plot_y[:10], c='orchid', alpha=0.7,
               label='data' if first else '__no_label__', zorder=2)
    ax[1].plot(plot_x[9:11], plot_y[9:11], '--', c='orchid', alpha=0.3,
               label='__no_label__', zorder=2)
    ax[1].plot(plot_x[10:20], plot_y[10:20], c='orchid', alpha=0.7,
               label='__no_label__', zorder=2)
    ax[1].plot(plot_x[19:21], plot_y[19:21], '--', c='orchid', alpha=0.3,
               label='__no_label__', zorder=2)
    ax[1].plot(plot_x[20:], plot_y[20:], c='orchid', alpha=0.7,
               label='__no_label__', zorder=2)
    first=False

# Model
ax[1].plot(t_p, solution_pulse[t<=t_end_p,0], '-', color='k', lw=2.0, label='model', zorder=3)

# Cosmetics
ax[1].set_ylabel('$sim$ expression', fontsize=14, fontname='Arial', labelpad=0)
ax[1].set_xlabel('time [min]', fontsize=14, fontname='Arial')
ax[1].tick_params(axis='both', which='major', labelsize=14)
[tick.set_fontname('Arial') for tick in ax[1].get_xticklabels()]
[tick.set_fontname('Arial') for tick in ax[1].get_yticklabels()]
ax[1].set_xlim([-0.5, 40.5])

handles, labels = ax[1].get_legend_handles_labels()
order = [0,1,2]
ax[1].legend([handles[idx] for idx in order[:len(handles)]], 
             [labels[idx] for idx in order[:len(labels)]],
             scatterpoints=3, loc=2, fontsize=12,
             prop={'family':'Arial', 'size':12})


# FINALIZE

ax[0].set_ylim(ax[1].get_ylim())

plt.tight_layout()
plt.savefig(r'../Figures/3_DE.pdf')
plt.show()

In [ ]:
### Phase space

# Settings
dt_scale = 0.5
on_strength = 0.8
A_range  = np.linspace(0, 0.8, 15)
B_range  = np.linspace(0, 0.006, 15)

# Generate grid
A_all, B_all = np.meshgrid(A_range, B_range)
A_all = A_all.flatten()
B_all = B_all.flatten()

# Run simulations on the grid
dA_all_on, dB_all_on = ode_model((A_all, B_all), 0, optimal_params, lambda t : on_strength)
dA_all_off, dB_all_off = ode_model((A_all, B_all), 0, optimal_params, lambda t : 0.0)

# Prep figure
fix, ax = plt.subplots(1, 2, figsize=(14,7))

# Show the phase space
ax[0].quiver(A_all, B_all, dA_all_off*dt_scale, dB_all_off*dt_scale,
             angles='xy', scale_units='xy', scale=1, 
             color='black', alpha=0.3, linewidth=0.5)
ax[0].scatter(A_all, B_all, c='darkred', s=12, alpha=1.0)
ax[0].set_title("Input Off (0.0)")

ax[1].quiver(A_all, B_all, dA_all_on*dt_scale, dB_all_on*dt_scale,
             angles='xy', scale_units='xy', scale=1, 
             color='black', alpha=0.3, linewidth=0.5)
ax[1].scatter(A_all, B_all, c='blue', s=12, alpha=1.0)
ax[1].set_title("Input On ("+str(on_strength)+")")

# Recreate dynamic simulations
output_cont = odeint(ode_model, ini, t, args=(optimal_params, md.input_functions.input_continuous_model))
output_pulse = odeint(ode_model, ini, t, args=(optimal_params, md.input_functions.input_pulsatile_model))

# Show the dynamic sim traces in phase space
for axis in ax:
    axis.plot(output_cont[:,0],  output_cont[:,1], c='teal', lw=2, label='continuous')
    axis.plot(output_cont[0,0],  output_cont[0,1],  c='teal', marker='o')
    axis.plot(output_cont[-1,0], output_cont[-1,1], c='teal', marker='s')

    axis.plot(output_pulse[:,0],  output_pulse[:,1], c='orchid', lw=2, label='pulsatile')
    axis.plot(output_pulse[0,0],  output_pulse[0,1],  c='orchid', marker='o')
    axis.plot(output_pulse[-1,0], output_pulse[-1,1], c='orchid', marker='s')
    
# Cosmetics
ax[0].set_xlabel('A', fontsize=16)
ax[1].set_xlabel('A', fontsize=16)
ax[0].set_ylabel('B', fontsize=16)

# Finish
plt.tight_layout()
plt.show()

### Perturbations

- KO of B (putative hairless KO); `k_AB=0.0`
- KO of hypothetical inhibitor of A activation (putative hairless KO); `k_IA = k_IA(wt) * 10.0`
- Stabilize A (putative twist OE); `k_BA=k_BA(wt) / 10.0` (alternatively, use `k_BA=0.0`)

In [ ]:
### Perturbations

# Parameters: k_IA, k_BA, k_Ad, k_AB, k_Bd
perturbations = {'Wild-Type'                : optimal_params,
                 'KO of Inhibitor (B)'      : [p if i!=3 else 0.0 for i,p in enumerate(optimal_params)],
                 'KO of Inhibitor (~k_IA)'  : [p if i!=0 else p*10.0 for i,p in enumerate(optimal_params)],
                 'Stabilized Activator (A)' : [p if i!=1 else p/10.0 for i,p in enumerate(optimal_params)]}

# Figure colors
linecolors = {'Wild-Type'                : 'k',
              'KO of Inhibitor (B)'      : 'firebrick',
              'KO of Inhibitor (~k_IA)'  : 'firebrick',
              'Stabilized Activator (A)' : 'darkblue'}

# Prep figure
fig, axes = plt.subplots(4, 1, figsize=(5, 5 /3*4), sharex=True, sharey=True)

# Run simulations and plot
for pert, ax in zip(perturbations, axes):
    
    # Initial conditions
    A = 0; B = 0; ini = (A, B)
    
    # Time
    dt = 0.1; t = np.arange(0, 40, dt)
    
    # Run sim
    solution = odeint(ode_model, ini, t, args=(perturbations[pert], 
                                               md.input_functions.input_continuous_model))
    solution *= scale_factor
    output_signal= solution[:,0]
    
    # Plot
    ax.plot(t, output_signal, '-', color=linecolors[pert], lw=2.0)
    ax.tick_params(axis='both', which='major', labelsize=13)
    ax.set_yticks([10, 30, 50, 70, 90])
    ax.set_ylim([0, 110])

# Cosmetics
fig.text(-0.02, 0.56, '$sim$ expression', va='center', rotation='vertical', fontsize=14)
axes[-1].set_xlabel('time [min]', fontsize=14)

# Finalize
plt.tight_layout()
plt.savefig(r'../Figures/4_A.pdf')
plt.show()